In [ ]:
# Update sklearn to prevent version mismatches
#!pip install sklearn --upgrade

In [ ]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
#!pip install joblib

In [1]:
import pandas as pd
import numpy as np

Import our white and red wine data from the csvs

# Read the CSV and Perform Basic Data Cleaning

In [3]:
red_wine_df = pd.read_csv("winequality-red.csv", sep=';')
white_wine_df = pd.read_csv("winequality-white.csv", sep=';')
white_wine_df.info()
red_wine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
fixed acidity           4898 non-null float64
volatile acidity        4898 non-null float64
citric acid             4898 non-null float64
residual sugar          4898 non-null float64
chlorides               4898 non-null float64
free sulfur dioxide     4898 non-null float64
total sulfur dioxide    4898 non-null float64
density                 4898 non-null float64
pH                      4898 non-null float64
sulphates               4898 non-null float64
alcohol                 4898 non-null float64
quality                 4898 non-null int64
dtypes: float64(11), int64(1)
memory usage: 459.3 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
fixed acidity           1599 non-null float64
volatile acidity        1599 non-null float64
citric acid             1599 non-null float64
residual sugar          1599 non-null float64

## Balanced (Normalized) Data for White Wine Data (1599 data points) and Red Wine Data (1599 data points) 

In [4]:
#take a random sample of white wines to match the red wine data
random_white_df = white_wine_df.sample(1599,random_state=42).reset_index(drop=True)

Only balanced values are baseline condition we can use:

In [5]:
#if we want our only features of the model to have balanced white wine and red wine data, we use this data frame:
baseline_red_df = red_wine_df.copy()
baseline_white_df = random_white_df.copy()
only_balanced_df = pd.concat([baseline_red_df, baseline_white_df])
#only_balanced_df

In [6]:
only_balanced_df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.7,0.34,0.31,16.4,0.051,20.0,146.0,0.99834,3.06,0.54,9.1,5
1595,9.3,0.31,0.49,1.3,0.042,34.0,147.0,0.99480,3.11,0.46,9.8,5
1596,6.4,0.17,0.27,6.7,0.036,88.0,223.0,0.99480,3.28,0.35,10.2,6
1597,7.5,0.29,0.36,15.7,0.050,29.0,124.0,0.99680,3.06,0.54,10.4,5


In [ ]:
balance_gp = only_balanced_df.groupby('quality')
balance_gp.quality.count()

In [ ]:
only_balanced_df.groupby('quality').count()
# range_balanced_df.groupby('quality').count()

In [29]:
#Suppose we want to take specific ranges in only_balanced_df, which does not include colors
range_balanced_df = only_balanced_df.loc[\
(only_balanced_df["quality"] <= 8 ) & \
(only_balanced_df["quality"] >= 3 )  \
] 


In [30]:
range_balanced_df.groupby('quality').count()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
quality,,,,,,,,,,,
3,17,17,17,17,17,17,17,17,17,17,17
4,97,97,97,97,97,97,97,97,97,97,97
5,1146,1146,1146,1146,1146,1146,1146,1146,1146,1146,1146
6,1363,1363,1363,1363,1363,1363,1363,1363,1363,1363,1363
7,502,502,502,502,502,502,502,502,502,502,502
8,73,73,73,73,73,73,73,73,73,73,73


In [31]:
#if we want to include wine colors as features in our model:
color_red_df = red_wine_df.copy()
color_white_df = random_white_df.copy()
color_red_df["color"] = "red"
color_white_df["color"] = "white"
colors_df = pd.concat([color_red_df, color_white_df])

Added Color with get dummies (Or in case we want to remove it)

In [32]:
#if we only want to use the balanced data and include the colors of wine:
#0 is red, 1 is white
dummy_colors_df = pd.get_dummies(colors_df, columns=['color'])
dummy_drop_colors_df = dummy_colors_df.drop(columns=["color_red"]).copy()
dummy_rename_colors_df = dummy_drop_colors_df.rename(columns={"color_white":"colors"})
dummy_rename_colors_df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,colors
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,0
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.7,0.34,0.31,16.4,0.051,20.0,146.0,0.99834,3.06,0.54,9.1,5,1
1595,9.3,0.31,0.49,1.3,0.042,34.0,147.0,0.99480,3.11,0.46,9.8,5,1
1596,6.4,0.17,0.27,6.7,0.036,88.0,223.0,0.99480,3.28,0.35,10.2,6,1
1597,7.5,0.29,0.36,15.7,0.050,29.0,124.0,0.99680,3.06,0.54,10.4,5,1


If we want to hypertune further, and choose a specfic range of quality scores, so our model predicts a smaller range of quality scores

In [35]:
#For our model that includes our `colors` feature:
range_colors_df = dummy_rename_colors_df.loc[\
(dummy_rename_colors_df["quality"] <= 6 ) & \
(dummy_rename_colors_df["quality"] >= 3 )  \
] 
range_colors_df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,colors
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,0
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.7,0.34,0.31,16.4,0.051,20.0,146.0,0.99834,3.06,0.54,9.1,5,1
1595,9.3,0.31,0.49,1.3,0.042,34.0,147.0,0.99480,3.11,0.46,9.8,5,1
1596,6.4,0.17,0.27,6.7,0.036,88.0,223.0,0.99480,3.28,0.35,10.2,6,1
1597,7.5,0.29,0.36,15.7,0.050,29.0,124.0,0.99680,3.06,0.54,10.4,5,1


In [36]:
#To show which range we are using
range_colors_df.groupby('quality').count()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,colors
quality,,,,,,,,,,,,
3,17,17,17,17,17,17,17,17,17,17,17,17
4,97,97,97,97,97,97,97,97,97,97,97,97
5,1146,1146,1146,1146,1146,1146,1146,1146,1146,1146,1146,1146
6,1363,1363,1363,1363,1363,1363,1363,1363,1363,1363,1363,1363


Add buckets to given range of quality values

In [45]:
only_balanced_df.groupby('quality').count()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
quality,,,,,,,,,,,
3,17,17,17,17,17,17,17,17,17,17,17
4,97,97,97,97,97,97,97,97,97,97,97
5,1146,1146,1146,1146,1146,1146,1146,1146,1146,1146,1146
6,1363,1363,1363,1363,1363,1363,1363,1363,1363,1363,1363
7,502,502,502,502,502,502,502,502,502,502,502
8,73,73,73,73,73,73,73,73,73,73,73


### Moved quality values into buckets

Created bins for balanced data called: "terrible (0), mediocre (1), and great (2)"
The quality scores range from 0 to 4, 5 to 6, and 7 to 10. We will call them 0,1,2.

In [53]:
bin_balanced_df = only_balanced_df.copy()
bins = [0, 5, 6, 10]
group_names = ["terrible", "mediocre", "great"]
bin_balanced_df["bin_quality"] = pd.cut(bin_balanced_df["quality"], bins, labels=group_names)
bin_qual_df = bin_balanced_df.drop(columns="quality")
bin_qual_df
dummy_bins_df = pd.get_dummies(bin_qual_df, columns=['bin_quality'])
dummy_bins_df
#dummy_bins_df.sample(30)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,bin_quality_terrible,bin_quality_mediocre,bin_quality_great
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,1,0,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,1,0,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,1,0,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,0,1,0
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.7,0.34,0.31,16.4,0.051,20.0,146.0,0.99834,3.06,0.54,9.1,1,0,0
1595,9.3,0.31,0.49,1.3,0.042,34.0,147.0,0.99480,3.11,0.46,9.8,1,0,0
1596,6.4,0.17,0.27,6.7,0.036,88.0,223.0,0.99480,3.28,0.35,10.2,0,1,0
1597,7.5,0.29,0.36,15.7,0.050,29.0,124.0,0.99680,3.06,0.54,10.4,1,0,0


In [54]:
bin_qual_df.groupby("bin_quality").count()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
bin_quality,,,,,,,,,,,
terrible,1260,1260,1260,1260,1260,1260,1260,1260,1260,1260,1260
mediocre,1363,1363,1363,1363,1363,1363,1363,1363,1363,1363,1363
great,575,575,575,575,575,575,575,575,575,575,575


Now we want to add colors with the convert quality scores to bins to see what happens

In [55]:
bins_colors_df = dummy_rename_colors_df.copy()
bins = [0, 5, 6, 10]
group_names = [1, 2, 3]
bins_colors_df["bin_quality"] = pd.cut(bins_colors_df["quality"], bins, labels=group_names)
new_bins_colors_df = bins_colors_df.drop(columns="quality")
new_bins_colors_df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,colors,bin_quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,0,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,0,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,0,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,0,2
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.7,0.34,0.31,16.4,0.051,20.0,146.0,0.99834,3.06,0.54,9.1,1,1
1595,9.3,0.31,0.49,1.3,0.042,34.0,147.0,0.99480,3.11,0.46,9.8,1,1
1596,6.4,0.17,0.27,6.7,0.036,88.0,223.0,0.99480,3.28,0.35,10.2,1,2
1597,7.5,0.29,0.36,15.7,0.050,29.0,124.0,0.99680,3.06,0.54,10.4,1,1


In [56]:
new_bins_colors_df.groupby("bin_quality").count()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,colors
bin_quality,,,,,,,,,,,,
1,1260,1260,1260,1260,1260,1260,1260,1260,1260,1260,1260,1260
2,1363,1363,1363,1363,1363,1363,1363,1363,1363,1363,1363,1363
3,575,575,575,575,575,575,575,575,575,575,575,575


# Unbalanced (Denormalized) Data for White Wine Data (4898 data points) and Red Wine Data (1599 data points) 

In [ ]:
only_balanced_df

# Select your features (columns)

In [ ]:
# Set features. Drop quality, y value. 
X = new_bins_colors_df.drop(columns=['binquality'])
X.head()

# Create a Train Test Split

Use `quality` for the y values

In [ ]:
from sklearn.model_selection import train_test_split
#80% train, 20% test. y is this one column
y = only_balanced_df['quality']
#random state 42 will have same picks for x test and y test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
X_train

# Train the Model using Random Forest 



In [ ]:
#Train the model
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)

In [ ]:
#Test the model
from sklearn.metrics import confusion_matrix, classification_report
predictions = rfc.predict(X_test)
print(classification_report(y_test, predictions))

# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'get_dummies_four_bins.sav'
joblib.dump(predictions, filename)

In [9]:
X = only_balanced_df.copy()
X.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [10]:
y = X.pop("quality")

In [12]:
y

0       5
1       5
2       5
3       6
4       5
       ..
1594    5
1595    5
1596    6
1597    5
1598    6
Name: quality, Length: 3198, dtype: int64

In [16]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import roc_auc_score

model = RandomForestRegressor(100,oob_score=True, random_state=42)
model.fit(X,y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=True,
                      random_state=42, verbose=0, warm_start=False)

In [20]:
model.oob_score_

0.4417595367245897

In [28]:
y_oob = model.oob_prediction_
roc_auc_score(y, y_oob)

ValueError: multi_class must be in ('ovo', 'ovr')

In [13]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
predictions = rfc.predict(X_test)
print(classification_report(y_test, predictions))